# Introdução à Inteligência Artificial 2021/2022

## Problemas de Satisfação de Restrições - Constraint Satisfaction Problems (CSP)

## Conteúdo

* Revisão da definição de CSP
* Exemplo de modelização de um problema com 3 variáveis
* A resolução de um CSP como uma procura num grafo
* O algoritmo de base - profundidade primeiro com retrocesso (BACKTRACKING-SEARCH), sem inferências ou heurísticas ou pré-processamento.
* Relevância da ordem inicial dos domínios
* Forward checking
* Procura com Retrocesso usando a inferência Forward Checking
* Teste de Consistência dos arcos
* Forçar a consistência dos arcos
* O algoritmo AC3
* Pré-processamento com o AC3 + procura em retrocesso com ou sem forward checking
* Procura com AC3
* Heurística MRV
* Combinação de inferências e MRV

## Solução de CSPs

**A resolução de um CSP tem 2 passos**:

**Passo 1)** Formulação do problema (definir variáveis, domínios, restrições)

**Passo 2)** Resolução do problema (usar algoritmo de procura)

Na última aula de laboratório dedicámo-nos à formulação e nesta iremos dedicarmo-nos à resolução dos CSPs.

Nesta aula vamos formular e resolver Problemas de Satisfação de Restrições - **Constraint Satisfaction Problems (CSP)**.


## Definição de um CSP
, Relembrando, o CSP é caracterizado por 3 componentes:
- **Variáveis** - para as quais se quer encontrar uma afectação (*assigment*) completa e consistente (solução); podem ser discretas ou contínuas.
- **Domínios** - indicam os valores possíveis das variáveis; podem ser finitos ou infinitos.
- **Restrições** - especificam combinações possíveis de valores para subconjuntos de variáveis; podem ser 1) unárias por exemplo <$(X_1)$: $X_1=5$>, 2) binárias, por exemplo <$(X_1$,$X_2)$: $X_1=X_2$>, ou de ordem superior, por exemplo <$(X_1,X_2,X_3)$, alldiff($X_1,X_2,X_3$)>`.

## Exemplo de modelização de um problema com 3 variáveis
Relembremos a modelização. Consideremos que temos 3 variáveis, $A$, $B$ e $C$, cada uma com 3 valores possíveis: {1,2,3} e as restrições obrigam a que $A > B > C$. Queremos encontrar os valores para as variáveis de modo a satisfazer as restrições.

Vamos modelizar o problema de acordo com o módulo `csp`, criando uma função **CSP_abc()** que vai devolver uma instância da classe **CSP**.

In [1]:
from csp_v2 import *

### Criação de um CSP usando a classe **CSP**

Esta classe permite definir um CSP assumindo que os domínios das variáveis são finitos. Para definir um CSP concreto é necessário definir as variáveis, os domínios, o grafo de restrições bem como as restrições, e todos serão depois usados para criar um objecto da classe CSP.

Neste contexto, o construtor **`def __init__(self, variables, domains, neighbors, constraints)`** tem os seguintes argumentos:
  
* **`variables`** : Lista de variáveis (strings ou inteiros).

* **`domains`** : Dicionário com elementos do tipo `{var:[valor, ...]}`.

* **`neighbors`**: Dicionário com elementos do tipo `{var:[var,...]}` em que cada variável var (chave) tem como valor uma lista das variáveis com as quais tem restrições (vizinhos no grafo de restrições), que define o grafo de restrições.

* **`constraints`** : Função do tipo `f(A, a, B, b)` que devolve `True` se os vizinhos `A` e `B` satisfazem a restrição quando têm valores `A=a` e `B=b`. 

In [2]:
def CSP_abc ():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema ABC
    """
    
    #Definir Variáveis
    variaveis_abc = ['A', 'B','C']
    
    """Definir Domínios
    Devolve um dicionario com o mesmo domínio [1,2,3] para as 3 variáveis.
    """   
    dominios_abc = {}
    for v in variaveis_abc :
        dominios_abc[v] = [1,2,3]
 
    #Definir Vizinhos    
    # parse_neighbors trata da simetria. do not worry
    vizinhos_abc = parse_neighbors('A : B; B: C')

    #Definir função que verifica restrições binárias
    def restricoes_abc(X, a, Y, b) :
        """
        A implementação em CSP assume que as restrições são binárias.
        Retorna True se (X=a,Y=b) satisfaz as restrições entre X e Y ou se não há restrição entre X e Y
        """
        out = True
        if (X == 'A' and Y=='B') or (X == 'B' and Y=='C'):
            out = a > b
        elif (X == 'B' and Y == 'A') or (X == 'C' and Y == 'B'):
            out = a < b
            
        return out
        
    return CSP(variaveis_abc, dominios_abc, vizinhos_abc, restricoes_abc)

Vamos agora criar um objecto **CSP** usando a função **CSP_abc()** que implementámos em cima:

In [3]:
p = CSP_abc()

E verificar os valores das variáveis, dos domínios e dos vizinhos (grafo de restrições):

In [4]:
print("Variáveis = ", p.variables)

Variáveis =  ['A', 'B', 'C']


In [5]:
print("Domínios = ", p.domains)

Domínios =  {'A': [1, 2, 3], 'B': [1, 2, 3], 'C': [1, 2, 3]}


In [6]:
p.domains['B']

[1, 2, 3]

In [7]:
print("Vizinhos = ", p.neighbors)

Vizinhos =  defaultdict(<class 'list'>, {'A': ['B'], 'B': ['A', 'C'], 'C': ['B']})


Vamos testar se a função **restricoes_abc()** foi bem implementada, verificando algumas afectações a pares de variáveis, confirmando que a função retorna *True*, se os dois pares variável-valor, passados como argumentos, satisfazem as restrições e *False*, caso contrário.

In [8]:
# Restrição A > B
print(p.constraints('A', 2, "B", 3))
print(p.constraints('A', 3, "B", 2))
print(p.constraints('B', 2, "A", 3))
print(p.constraints('B', 3, "A", 2))

False
True
True
False


In [9]:
# Restrição B > C
print(p.constraints('C', 2, 'B', 3))
print(p.constraints('C', 3, 'B', 2))
print(p.constraints('B', 2, 'C', 3))
print(p.constraints('B', 3, 'C', 2))

True
False
False
True


### A resolução de um CSP como uma procura num grafo

* **Estado** - é um conjunto de variáveis com valores nos seus domínios (afectação/assignment); 

* uma **afectação** pode ser:

    1) global, se todas as variáveis tem valor, ou parcial, caso contrário; e
    
    2) consistente, se  satisfaz todas as restrições, ou inconsistente, caso contrário.


* **Procura** - é efectuada usando algoritmos que usam heurísticas genéricas em vez de heurísticas específicas do problema, como acontecia na resolução dos Problemas de Espaços de Estados (PEE).
* **Solução** - é uma afectação completa e consistente.

### O algoritmo de base - profundidade primeiro com retrocesso (Backtracking-Search)
Recorde o algoritmo de procura **BACKTRACKING-SEARCH**, que efectua procura em profundidade primeiro com retrocesso, em árvore, fazendo afectações a uma única e  mesma variável a cada nível da procura. Sempre que a última afectação provocar conflitos de restrições com as variáveis já afectadas dá-se o retrocesso. Quando todos os valores de uma variável falham, dá-se também um retrocesso para o próximo valor da variável anterior, se existir. Notem que a procura em profundidade depende muito da ordem das variáveis e da ordem dos valores do domínio.

Vamos agora usar o algoritmo **BACKTRACKING-SEARCH** para resolver o problema. 

Podem ver no `csp_v2.py` que os argumentos desse método por omissão são:

``` python
    def backtracking_search(csp,
                        select_unassigned_variable=first_unassigned_variable,
                        order_domain_values=unordered_domain_values,
                        inference=no_inference, verbose = False):
        
```
    manter a ordem das variáveis definida no objecto csp
    manter a ordem dos valores nos domínios
    não haver qualquer inferência
    e ausência de verbosidade (não imprime nada)

Vamos invocar a profundidade com retrocesso com os argumentos por omissão

In [10]:
r = backtracking_search(p)
print('Assignment p = ',r) 

Assignment p =  {'A': 3, 'B': 2, 'C': 1}


Tal como esperado a solução obtida é {'A': 3, 'B': 2, 'C': 1}. 

Por omissão não ecoa o comportamento do algoritmo de procura. Vamos colocar o `verbose=True` para seguirmos com detalhe todo o processo de resolução do CSP.

O que esperamos que aconteça?

* Primeiro $A$ fica com o primeiro valor do seu domínio ordenado, i.e. 1; 

* depois tenta afectar-se $B$, mas qualquer valor tentado para $B$ viola a restrição entre $A$ e $B$, forçando o retrocesso de $A$ para 2;

* a seguir $B$ já pode ficar com o valor 1 mas não há valor de $C$ que satisfaça $B$ > $C$, forçando o retrocesso em $B$ até esgotar todos os seus valores

* forçando o retrocesso de $A$ para $A$=3, tentando $B$=1, mas falhando e

* só depois $B$=2 e $C$=1, conseguindo uma solução consistente. 

O output `curr_domains` na primeira chamada de **backtracking_search()**, que é uma função recursiva, tem o valor `None` mas ficará com o valor de domains, mas não se vê isso no modo verbose. Não estranhem.

In [11]:
p = CSP_abc()
r = backtracking_search(p,verbose = True)
print('Assignment p = ',r) 

Curr_domains: None
Current assignment: {}

Next selected Var = A
Sorted domain left [1, 2, 3]
--Test  A 1
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {'A': [1], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Current assignment: {'A': 1}

Next selected Var = B
Sorted domain left [1, 2, 3]
--Test  B 1
---Conflict!! Backtrack
--Test  B 2
---Conflict!! Backtrack
--Test  B 3
---Conflict!! Backtrack
---All values tested for  B

---Backtrack on A = 1
Restored domains: {'A': [2, 3], 'B': [1, 2, 3], 'C': [1, 2, 3]}
--Test  A 2
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {'A': [2], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Current assignment: {'A': 2}

Next selected Var = B
Sorted domain left [1, 2, 3]
--Test  B 1
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {'A': [2], 'B': [1], 'C': [1, 2, 3]}
Current assignment: {'A': 2, 'B': 1}

Next selected Var = C
Sorted domain left [

### Exercício 1
Imagine que tem exactamente o mesmo problema mas que o domínio de $A$ é apenas formado por 1 e por 2. Aplique o `backtracking_search` com o modo verbose activo, depois de reformular.

In [12]:
# Solução do exercício 1
def CSP_abc_WithRest ():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema ABC
    """
    
    #Definir Variáveis
    variaveis_abc = ['A', 'B','C']
    
    """Definir Domínios
    Devolve um dicionario com o mesmo domínio [1,2,3] para B e C e com A com dominio [1,2]
    """   
    dominios_abc = {}
    for v in variaveis_abc :
        dominios_abc[v] = [1,2,3]
    dominios_abc['A'] = [1,2]
 
    #Definir Vizinhos    
    # parse_neighbors trata da simetria. do not worry
    vizinhos_abc = parse_neighbors('A : B; B: C')

    #Definir função que verifica restrições binárias
    def restricoes_abc(X, a, Y, b) :
        """
        A implementação em CSP assume que as restrições são binárias.
        Retorna True se (X=a,Y=b) satisfaz as restrições entre X e Y ou se não há restrição entre X e Y
        """
        out = True
        if (X == 'A' and Y=='B') or (X == 'B' and Y=='C'):
            out = a > b
        elif (X == 'B' and Y == 'A') or (X == 'C' and Y == 'B'):
            out = a < b
            
        return out
    return CSP(variaveis_abc, dominios_abc, vizinhos_abc, restricoes_abc)

p1 = CSP_abc_WithRest()
r = backtracking_search(p1,verbose = True)
print('Assignment p = ',r) 

Curr_domains: None
Current assignment: {}

Next selected Var = A
Sorted domain left [1, 2]
--Test  A 1
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {'A': [1], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Current assignment: {'A': 1}

Next selected Var = B
Sorted domain left [1, 2, 3]
--Test  B 1
---Conflict!! Backtrack
--Test  B 2
---Conflict!! Backtrack
--Test  B 3
---Conflict!! Backtrack
---All values tested for  B

---Backtrack on A = 1
Restored domains: {'A': [2], 'B': [1, 2, 3], 'C': [1, 2, 3]}
--Test  A 2
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {'A': [2], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Current assignment: {'A': 2}

Next selected Var = B
Sorted domain left [1, 2, 3]
--Test  B 1
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {'A': [2], 'B': [1], 'C': [1, 2, 3]}
Current assignment: {'A': 2, 'B': 1}

Next selected Var = C
Sorted domain left [1, 2, 

Como era esperado, não há solução! A procura foi em vão.

### Relevância da ordem inicial dos domínios
Notem que a escolha fixa da ordem dos valores do domínio é relevante no processo de procura. Por exemplo, poderíamos reescrever o modelo mas em que invertemos a ordem dos valores no domínio de modo a que fiquem por ordem decrescente ao longo de todo o processo de procura. Reparem que ao reconstruirmos o objecto da classe **CSP** apenas invertemos a lista correspondente aos valores dos domínios.

In [13]:
def CSP_abc_inv ():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema ABC
    """
    
    #Definir Variáveis
    variaveis_abc = ['A', 'B','C']
    
    """Definir Domínios
    Devolve um dicionario com os domínios com as variáveis do problema dos Golfers.
    """   
    dominios_abc = {}
    for v in variaveis_abc :
        dominios_abc[v] = [3,2,1]
 
    #Definir Vizinhos    
    # parse_neighbors trata da simetria. do not worry
    vizinhos_abc = parse_neighbors('A : B; B: C')

    #Definir função que verifica restrições binárias
    def restricoes_abc(X, a, Y, b) :
        """
        A implementação em CSP assume que as restrições são binárias.
        Retorna True se (X=a,Y=b) satisfaz as restrições entre X e Y ou se não há restrição entre X e Y
        """
        out = True
        if (X == 'A' and Y=='B') or (X == 'B' and Y=='C'):
            out = a > b
        elif (X == 'B' and Y == 'A') or (X == 'C' and Y == 'B'):
            out = b > a
            
        return out
        
    return CSP(variaveis_abc, dominios_abc, vizinhos_abc, restricoes_abc)

Vamos procurar então... com o modo verboso activo

In [14]:
p = CSP_abc_inv()
r = backtracking_search(p,verbose = True)
print('Assignment p = ',r)

Curr_domains: None
Current assignment: {}

Next selected Var = A
Sorted domain left [3, 2, 1]
--Test  A 3
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {'A': [3], 'B': [3, 2, 1], 'C': [3, 2, 1]}
Current assignment: {'A': 3}

Next selected Var = B
Sorted domain left [3, 2, 1]
--Test  B 3
---Conflict!! Backtrack
--Test  B 2
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {'A': [3], 'B': [2], 'C': [3, 2, 1]}
Current assignment: {'A': 3, 'B': 2}

Next selected Var = C
Sorted domain left [3, 2, 1]
--Test  C 3
---Conflict!! Backtrack
--Test  C 2
---Conflict!! Backtrack
--Test  C 1
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Assignment p =  {'A': 3, 'B': 2, 'C': 1}


Neste caso, a ordem invertida dos domínios decrementou o tempo de procura, particularmente porque se adaptou a um problema cujas restrições obrigavam a valores de C menores do que os de $B$ e de $A$ e valores de $B$ menores do que os de $A$. É claro que saber qual a ordem ideal de afectação das variáveis é uma incógnita para a maior parte dos problemas.

### Problema das N raínhas

<img src=".\imagens\Eight-queens-animation.gif" alt="Drawing" style="width: 180px;"/>

Considere o problema de colocar N raínhas num tabuleiro NxN de modo a que nenhuma se ataque. 
Na formulação das N raínhas temos como

* variáveis: os números de $0$ a $N-1$, correspondentes às linhas ($0$ a mais no topo e $N-1$ a mais no fundo).
* domínios: a lista de [0,1,...,N], correspondentes às colunas: 0, a mais à esquerda e N-1 a mais à direita.
* restrições: duas raínhas (sempre de linhas diferentes) não podem estar na mesma colunas nem nas mesmas diagonais. A vantagem de usarmos números para as variáveis é que podemos usá-las para os cálculos em que verificamos se não pertencem à mesma diagonal.
    
Note que os valores dos domínios e as variáveis são números de 0 a N-1. Esperemos que não se confundam.

A implementação da função que gera o construtor da classe CSP para este problema está na célula python seguinte:

In [15]:
def queen_constraint(A, a, B, b):
    """Constraint is satisfied (true) if A, B are really the same variable, or if they are not in the same row, down diagonal, or up diagonal."""
    return (a != b and A + a != B + b and A - a != B - b)

def NQueensCSP(n):
    """Make a CSP for the nQueens problem """
    variables=list(range(n))
    neighbors=dict()
    for v in variables:
        ns=list(range(n))
        ns.remove(v)
        neighbors[v]=ns
        domains=dict()
    for v in variables:
        ns=list(range(n))
        domains[v]=ns
        
    return CSP(variables, domains, neighbors, queen_constraint)

#### O problema com 4 raínhas

In [16]:
# criemos um problema com 5 raínhas e mostremos as variáveis, os domínios e os vizinhos:
num=4
q4 = NQueensCSP(num)
print('CSP para', num, 'raínhas:')
print('    As variáveis:',q4.variables)
print('    Os domínios:',q4.domains)
print('    Os vizinhos:',q4.neighbors)

CSP para 4 raínhas:
    As variáveis: [0, 1, 2, 3]
    Os domínios: {0: [0, 1, 2, 3], 1: [0, 1, 2, 3], 2: [0, 1, 2, 3], 3: [0, 1, 2, 3]}
    Os vizinhos: {0: [1, 2, 3], 1: [0, 2, 3], 2: [0, 1, 3], 3: [0, 1, 2]}


Vejemos a árvore de procura para as 4 raínhas:

<img src=".\imagens\bo9Iy.png" alt="Drawing" style="width: 250px;"/>

Invoquemos a função **backtracking_search()** com o modo verboso a off primeiro

In [17]:
r4=backtracking_search(q4)
print('Solução:',r4)

Solução: {0: 1, 1: 3, 2: 0, 3: 2}


Agora sigamos o processo da procura com o modo verboso ligado

In [18]:

r4 = backtracking_search(q4,verbose=True)
print('Solução:',r4)

Curr_domains: {0: [1], 1: [3], 2: [0], 3: [2]}
Current assignment: {}

Next selected Var = 0
Sorted domain left [1]
--Test  0 1
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {0: [1], 1: [3], 2: [0], 3: [2]}
Current assignment: {0: 1}

Next selected Var = 1
Sorted domain left [3]
--Test  1 3
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {0: [1], 1: [3], 2: [0], 3: [2]}
Current assignment: {0: 1, 1: 3}

Next selected Var = 2
Sorted domain left [0]
--Test  2 0
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Curr_domains: {0: [1], 1: [3], 2: [0], 3: [2]}
Current assignment: {0: 1, 1: 3, 2: 0}

Next selected Var = 3
Sorted domain left [2]
--Test  3 2
---UAU No Conflicts with already assigned variables!
No inference!
---Assigned!
Solução: {0: 1, 1: 3, 2: 0, 3: 2}


#### O problema com 5 raínhas
Depois de criarmos um problema com 5 raínhas, verifiquemos o tempo que demora a encontrar uma solução usando o caso base do **backtracking_search()**.

In [19]:
q5 = NQueensCSP(5)
import timeit
print('Retrocesso para',num, 'raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas')
start = timeit.default_timer()
r5 = backtracking_search(q5)
stop = timeit.default_timer()
time_exec = stop-start
print('Solução: ',r5)
print('Tempo de execução',time_exec)

Retrocesso para 4 raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas
Solução:  {0: 0, 1: 2, 2: 4, 3: 1, 4: 3}
Tempo de execução 0.00020619999999993421


O processo de procura corresponde a esta árvore:
<img src=".\imagens\tree_rainhas4.PNG" alt="Drawing" style="width: 600px;"/>


Experimentemos agora para o problema standard de 8 raínhas:

In [20]:
num=8
q = NQueensCSP(num)
import timeit
print('Retrocesso para',num, 'raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas')
start = timeit.default_timer()
r = backtracking_search(q)
stop = timeit.default_timer()
time_exec = stop-start
print('Assignment q = ',r)
print('Tempo de execução',time_exec)

Retrocesso para 8 raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas
Assignment q =  {0: 0, 1: 4, 2: 7, 3: 5, 4: 2, 5: 6, 6: 1, 7: 3}
Tempo de execução 0.005212800000000017


Aumentemos o tabuleiro para 20 raínhas:

In [21]:
num=20
q = NQueensCSP(num)
import timeit
print('Retrocesso para',num, 'raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas')
start = timeit.default_timer()
r = backtracking_search(q)
stop = timeit.default_timer()
time_exec = stop-start
print('Solução = ',r)
print('Tempo de execução',time_exec)

Retrocesso para 20 raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas


KeyboardInterrupt: 

### Exercício 2

Reformulem o problema das N raínhas de modo a que a `ordem das variáveis` não seja sempre a mesma, sendo gerada de forma aleatória durante a criação do CSP. Compare a resolução do problema usando o **backtracking_search()** para diferentes ordenações das variáveis, em que podem usar a função **random.shuffle()** para baralhar uma lista. Podem experimentar repetidas vezes para as 20 raínhas para comparar com a versão anterior com as variáveis ordenadas de forma crescente. Até podem experimentar para valores maiores de N.

### Exercício 3

Reformulem o problema das N raínhas de modo a que a `ordem dos valores` do domínio seja aleatório para cada uma das variáveis. Compare a resolução do problema usando o **backtracking_search()** para diferentes ordenações das variáveis, em que podem usar a função **random.shuffle()** para baralhar uma lista. Podem experimentar repetidas vezes para as 20 raínhas para comparar com as versões anteriores com as variáveis ordenadas de forma crescente e com as variáveis ordenadas de modo aleatório. Penso que podem ir mais longe e experimentar para 50 raínhas, por exemplo.

In [22]:
# Solução do exercício 2
def queen_constraint(A, a, B, b):
    """Constraint is satisfied (true) if A, B are really the same variable, or if they are not in the same row, down diagonal, or up diagonal."""
    return (a != b and A + a != B + b and A - a != B - b)

def NQueensCSP_shuffle_variables(n):
    """Make a CSP for the nQueens problem """
    variables=list(range(n))
    neighbors=dict()
    for v in variables:
        ns=list(range(n))
        ns.remove(v)
        neighbors[v]=ns
        domains=dict()
    for v in variables:
        ns=list(range(n))
        domains[v]=ns
        
    random.shuffle(variables)
        
    return CSP(variables, domains, neighbors, queen_constraint)


num=10
import timeit
q = NQueensCSP_shuffle_variables(num)
print('Retrocesso para',num, 'raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas')
start = timeit.default_timer()
r = backtracking_search(q)
stop = timeit.default_timer()
time_exec = stop-start
print('Assignment q = ',r)
print('Tempo de execução',time_exec)
    

Retrocesso para 10 raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas
Assignment q =  {3: 0, 4: 2, 0: 1, 7: 3, 2: 5, 8: 7, 9: 4, 5: 9, 1: 8, 6: 6}
Tempo de execução 0.0006192999999967697


In [23]:
# Solução do exercício 3
# Solução do exercício 2
def queen_constraint(A, a, B, b):
    """Constraint is satisfied (true) if A, B are really the same variable, or if they are not in the same row, down diagonal, or up diagonal."""
    return (a != b and A + a != B + b and A - a != B - b)

def NQueensCSP_shuffle_domain(n):
    """Make a CSP for the nQueens problem """
    variables=list(range(n))
    neighbors=dict()
    for v in variables:
        ns=list(range(n))
        ns.remove(v)
        neighbors[v]=ns
        domains=dict()
    for v in variables:
        ns=list(range(n))
        domains[v]=ns
        random.shuffle(domains[v])
        
    
        
    return CSP(variables, domains, neighbors, queen_constraint)

num=10
import timeit
q = NQueensCSP_shuffle_variables(num)
print('Retrocesso para',num, 'raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas')
start = timeit.default_timer()
r = backtracking_search(q)
stop = timeit.default_timer()
time_exec = stop-start
print('Assignment q = ',r)
print('Tempo de execução',time_exec)


Retrocesso para 10 raínhas, caso base, sem preprocessamento, sem inferência nem heurísticas
Assignment q =  {3: 0, 8: 1, 1: 3, 5: 6, 7: 7, 4: 2, 0: 5, 2: 8, 9: 4, 6: 9}
Tempo de execução 0.0025011999999975387


### Inferência Forward Checking
No forward checking, os domínios das outras variáveis, por afectar ainda, daí o nome "para a frente" (forward), vão ser filtrados dos valores que entram em conflito com a última afectação.
Iremos ilustrar o forward checking quando falha e quando sucede depois de uma afectação, tendo em conta as afectações anteriores e em conjunto com a procura em profundidade com retrocesso.

#### Quando o Forward Checking falha
Suponhamos que a primeira variável a ser afectada é $A$ e que vamos começar por atribuir-lhe o valor de 1. Removem-se do domínio de $A$ os outros dois valores mas como se diz na cozinha reservam-se para o caso de tudo falhar e termos de repôr os domínios originais. 

Iremos propagar essa afectação às restantes variáveis ainda sem afectação, todas ainda com o domínio {1,2,3} e veremos que o $B$ não tem nenhum valor menor do que $A$ e falha. Todos os valores do domínio de $B$ foram removidos porque chocaram com $A$=1 e foram também reservados. No final, depois da falha do forward checking todos os domínios foram restaurados.
Vamos ilustrar em python, fazendo uso de um conjunto de funções.

    O atributo curr_domains() guarda o domínio corrente das variáveis.
    A variável removals é a lista onde se colocam os pares (var,valor) que se reservaram


Comecemos por criar uma instância do CSP $A>B>C$ e suponhamos que $A$=1.

In [24]:
p = CSP_abc()
p.curr_domains=p.domains
print("Domínios:",p.curr_domains)
print('Suponhamos que A toma o valor de 1')
removals=p.suppose('A',1)
print('Teremos de remover do domínio os outros valores do domínio de A:',removals)
print(p.curr_domains)

Domínios: {'A': [1, 2, 3], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Suponhamos que A toma o valor de 1
Teremos de remover do domínio os outros valores do domínio de A: [('A', 2), ('A', 3)]
{'A': [1], 'B': [1, 2, 3], 'C': [1, 2, 3]}


Agora apliquemos o forward checking, 

In [25]:
print('Apliquemos o Forward Checking')
print('Sucede o forward checking?',forward_checking(p, 'A', 1, {}, removals, False))
print("Domínios após forward checking = ", p.curr_domains)

Apliquemos o Forward Checking
Sucede o forward checking? False
Domínios após forward checking =  {'A': [1], 'B': [], 'C': [1, 2, 3]}


O forward checking falha logo que uma das variáveis fique com o domínio vazio. Notem que ao invocar o forward checking passa-se uma variável, **removals**, que muda com a execução da função - são feitos *appends* no interior do **forward_checking()** e essa variável sai eventualmente mudada, mas não sai pelo canal do return. (Passagem por variável)

Ao falhar, se quiseremos podemos restaurar os domínios originais (antes da filtragem) de modo a podermos retroceder se assim o quisermos. Ilustremos o uso de **restore()**:

In [26]:
print('Os que foram removidos, mas que afinal terão de ser restaurados',removals)
print('Vamos restaurar')
p.restore(removals)
print("Domínios:",p.curr_domains)

Os que foram removidos, mas que afinal terão de ser restaurados [('A', 2), ('A', 3), ('B', 1), ('B', 2), ('B', 3)]
Vamos restaurar
Domínios: {'A': [1, 2, 3], 'B': [1, 2, 3], 'C': [1, 2, 3]}


#### Quando o Forward Checking sucede
Vamos agora ilustrar um caso de sucesso do forward checking. Suponhamos que em vez de atribuirmos a $A$ o valor de 1 atribuímos o 2.

In [27]:
print("Domínios:",p.curr_domains)
print('Suponhamos que A toma o valor de 2')
removals=p.suppose('A',2)
print('Teremos de remover do domínio os outros valores do domínio de A:',removals)
print(p.curr_domains)
print('Apliquemos o Forward Checking')
print('Sucede o forward checking?',forward_checking(p, 'A', 2, {}, removals, False))
print("Domínios após forward checking = ", p.curr_domains)

Domínios: {'A': [1, 2, 3], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Suponhamos que A toma o valor de 2
Teremos de remover do domínio os outros valores do domínio de A: [('A', 1), ('A', 3)]
{'A': [2], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Apliquemos o Forward Checking
Sucede o forward checking? True
Domínios após forward checking =  {'A': [2], 'B': [1], 'C': [1, 2, 3]}


Notem que, como $B$ tem de ser menor do que $A$, $B$ só pode ter o valor de 1 quando $A$ é afectado com o 2, removendo-se 2 e 3 do domínio de $B$.
$C$ não é afectado porque não há restrições entre $A$ e $C$.

Podemos ver os efeitos do forward checking sobre a versão em que os valores do domínio estão por ordem decrescente e afectamos $A$ a 3.

In [28]:
p = CSP_abc_inv()
print("Domínios:",p.curr_domains)
print('Suponhamos que A toma o valor de 3')
removals=p.suppose('A',3)
print('Teremos de remover do domínio os outros valores do domínio de A:',removals)
print(p.curr_domains)
print('Apliquemos o Forward Checking')
print('Sucede o forward checking?',forward_checking(p, 'A', 3, {}, removals, False))
print("Domínios após forward checking = ", p.curr_domains)

Domínios: None
Suponhamos que A toma o valor de 3
Teremos de remover do domínio os outros valores do domínio de A: [('A', 2), ('A', 1)]
{'A': [3], 'B': [3, 2, 1], 'C': [3, 2, 1]}
Apliquemos o Forward Checking
Sucede o forward checking? True
Domínios após forward checking =  {'A': [3], 'B': [2, 1], 'C': [3, 2, 1]}


### Procura com Retrocesso usando a inferência Forward Checking

Como vimos, o **FORWARD-CHECKING** permite detectar falhas antecipadamente: colocamos o argumento inference da função com o valor forward_checking. Permite também restringir os domínios das outras variáveis por afectar.

Procura com **FORWARD-CHECKING**: propaga para a frente (reduzindo os domínios das variáveis ainda por afectar) após cada afectação. Retrocede quando alguns dos domínios filtrados fica vazio.

Vamos aplicar a pesquisa em profundidade com retrocesso usando como inferência o forward checking. Vamos fazê-lo com o primeiro problema, $A>B>C$ com domínio {1,2,3} para todas as variáveis, a classe **CSP_abc()**. O resultado é o mesmo, com ou sem inferência mas a procura é menor como se pode observar.

In [ ]:
p = CSP_abc()
r = backtracking_search(p,inference = forward_checking)
print('Afectações = ',r)

Agora, a versão com `verbose = True`. Percam um pouco de tempo a seguir o rasto do processo de procura.

In [ ]:
p = CSP_abc()
r = backtracking_search(p,inference = forward_checking, verbose = True)
print('Afectações = ',r) 

Vejemos o que se passa com o caso do modelo com os domínios com os valores em modo decrescente.

In [ ]:
p = CSP_abc_inv()
r = backtracking_search(p,inference = forward_checking, verbose = True)
print('Afectações = ',r) 

### Usando a Procura com Retrocesso + forward checking para resolver o problema das 4 raínhas

Podemos ver o que acontece quando se aplica o forward-ckecking ao problema das 4 raínhas com as variáveis ordenadas de cima para baixo (ordem das variáveis: $0$, $1$, $2$ e $3$) e com as casas da esquerda para a direita (ordem dos domínios 0, 1, 2 e 3$).
Na árvore os valores dos domínios filtrados devido à inferência e afectação aparecem com x. 

<img src=".\imagens\tree_rainhas4-forwardCheck.PNG" alt="Drawing" style="width: 400px;"/>


#### Exercício 4
Compare em termos do tempo que demora a encontrar a solução a procura em profundidade com retrocesso simples e a versão com filtragem dos domínios por forward checking para problema das N raínhas. Teste vários valores de N, tendo em atenção que para valores elevados de N pode demorar muito tempo em qualquer dos 2 casos. Se quiserem ir mais longe podem também comparar as versões de domínios iniciais aleatórios com e sem inferência por forward checking.

In [ ]:
## Resolução exercício 4



### A Consistência dos Arcos
Um arco dirigido $X_i\rightarrow X_j$ é considerado como consistente se para todos os valores do domínio de $X_i$ existe pelo menos um valor do domínio de $X_j$ que satisfaz a restrição que envolve $X_i$ e $X_j$. Só faz sentido verificar a consistência dos arcos que estejam envolvidos em restrições.

A consistência dos arcos é verificada através da função **consistent()** que recebe 3 inputs: csp (uma instância de *CSP*), $X_i$ e $X_j$ (as duas variáveis correspondentes ao arco $X_i\rightarrow X_j$. 

```python
def consistent(csp,Xi,Xj):
    """is the arc xi->sj consistent?"""
    for x in csp.curr_domains[Xi][:]:
        if not any(csp.constraints(Xi, x, Xj, y) for y in csp.curr_domains[Xj]):
            return False
    return True
```

Regressemos ao CSP $A>B>C$ e testemos a consistência do arco $A \rightarrow B$:

In [ ]:
p = CSP_abc()
p.curr_domains=p.domains
print('Domínios actuais são os iniciais:',p.curr_domains)
print('É consistente o arco A->B?',consistent(p,'A','B'))

Vamos ver se o arco inverso $B \rightarrow A$ é consistente

In [ ]:
print('É consistente o arco B->A?',consistent(p,'B','A'))

Eis uma versão verbosa da função **consistent()** a que chamámos de **consistent_show()**

In [ ]:

def any_satisfies_show(csp,Xi,x,Xj,verbose=False):
    """Does any value in domain of Xj satisfies the constraint with Xi=x"""
    if verbose:
            print('Verificando',Xi,'=',x)
    for y in csp.curr_domains[Xj]:
        if verbose:
            print('       Verificando',Xj,'=',y)
        res=csp.constraints(Xi, x, Xj, y)
        if verbose:
            print('       Satisfaz restrição?',res)
        if res:
            return True
    print('       Todos falharam!')
    return False 
    
def consistent_show(csp,Xi,Xj,verbose=False):
    """is the arc xi->sj consistent? with or withou verbose mode"""
    for x in csp.curr_domains[Xi][:]:
        if not any_satisfies_show(csp,Xi,x,Xj,verbose):
            return False
    return True

Vamos invocá-la

In [ ]:
p = CSP_abc()
p.curr_domains=p.domains
print('Domínios actuais são os iniciais:',p.curr_domains)
print('Versão normal do A>B>C')
print('É consistente o arco B->A?')
consistent_show(p,'B','A',True)

#### Forçando a consistência de um arco
Para forçarmos a consistência de um arco $A\rightarrow B$ teremos de remover do domínio de $A$ todos os valores para os quais não há qualquer valor no domínio de $B$ que satisfaça a restrição associada ao arco.
A função que o faz é **revise()**

```python
def revise(csp, Xi, Xj, removals):
    "Return true if we remove a value."
    revised = False
    for x in csp.curr_domains[Xi][:]:
        # If Xi=x conflicts with Xj=y for every possible y, eliminate Xi=x
        if all(not csp.constraints(Xi, x, Xj, y) for y in csp.curr_domains[Xj]):
            csp.prune(Xi, x, removals)
            revised = True
    return revised
```

O input removals tem o valor vazio inicialmente por omissão e sai com os valores do domínio que terão de ser retirados no caso de a função ser bem sucedida, i.e., nenhum domínio vazio. o método **prune()** elimina do domínio $X_{i}$ o valor x e insere-o em removals.

In [ ]:
p = CSP_abc_inv()
p.curr_domains=p.domains
print('\n\nVersão invertida A>B>C')
print('Os domínios actuais são os iniciais:',p.curr_domains)
remove=None
print('Força consistência do arco B->A?',revise(p,'B','A',remove))
print('Os domínios actuais são:',p.curr_domains)

### Exercício 5
Confirme a consistência do arco formado pela raínha na linha $0$ e a raínha na linha $1$ para o estado inicial do problema das 4 raínhas, i.e. sem afectações. Não se esqueça de colocar o `curr_domain = domain`. Pode utilizar a versão verbosa da função consistent() para perceber que 
   
   
    quando a raínha da primeira linha, no topo (0) está na coluna da esquerda (0)
       a raínha da segunda linha (1) não pode estar nem na primeira nem na segunda coluna mas pode estar na terceira.
    quando a raínha da primeira linha está na segunda coluna (1) a partir da esquerda,
       a raínha da segunda linha (1) não pode estar nem na primeira, nem na segunda nem na terceira colunas, mas pode estar na quarta.
    quando a raínha da primeira (0) linha  está na 3ª coluna (2)
       a raínha da segunda linha (1) pode estar na primeira coluna (0)
    quando a raínha da primeira linha está na 4a coluna (3)
       a raínha da segunda linha (1) pode estar na primeira coluna (0)
   

In [ ]:
# solução do 5



### Pré-processamento usando AC3

O algoritmo AC3 verifica a consistência dos arcos e vai para além disso, forçando essa consistência alterando os domínios das variáveis. Se por acaso uma das variáveis ficar com o seu domínio vazio a função falha retornando `False`.

Esperando que seja não uma experiência enjoativa mas enriquecedora este vai e vém entre as N raínhas e $A>B>C$...

#### O algoritmo AC3
Vamos verificar para o problema $A>B>C$, que abriu este guião, que a utilização do AC3 reduz o número de valores possíveis no domínio de algumas variáveis, visto que estes são incompatíveis com as restrições.
Aliás, reduz de tal maneira que até encontra a solução sem ser necessário executar a procura.

Relembremos que no AC3 temos uma pilha de arcos e vamos forçando a consistência de cada arco, que é retirado da pilha, e se chegarmos a um domínio vazio, falha. Notem que em alguns casos, arcos já verificados terão de ser de novo colocados para o topo da pilha.

In [ ]:
#Execução do AC3
p = CSP_abc()
p_ac3 = AC3(p)
print("Domínios após AC3 = ", p_ac3.curr_domains)

Vamos executar o algoritmo AC3 em modo verbose para verificar o que acontece nos diferentes passos do algoritmo:

In [ ]:
#Execução do AC3
p = CSP_abc()
p_ac3 = AC3(p,verbose=True)
p_ac3.curr_domains

#### Exercício 6

Aplique o AC3 ao problema das 4 raínhas na situação de afectação vazia. Porque é que os domínios iniciais não mudam?

In [ ]:
# Solução do 6



#### Exercício 7
Aplique o AC3 ao problema das 4 raínhas na situação de afectação da variável $0$ com o valor 0 (raínha da coluna da esquerda está na linha de baixo). Como irá confirmar, não conseguiremos forçar a consistência dos arcos para o caso da primeira raínha (linha de topo) ficar na coluna da esquerda.

Note que após a criação da instância do CSP é necessário invocar **suppose(0,0)** para que os domínio da variável $0$ fique apenas com o valor 0.

In [ ]:
# solução do 7º



#### Exercício 8
Para o problema das 25 raínhas compare o tempo de execução usando retrocesso com "forward checking" com e sem pré-filtragem pela consistência dos arcos. 

In [ ]:
## Resolução do exercício 8



### Utilização de heurísticas no BACKTRACKING-SEARCH (MRV)

`MINIMUM-REMAINING-VALUES (MRV)` - heurística que escolhe a variável com o menor número de valores no domínio- nesta implementação em caso de empate escolhe-se a próxima var de modo aleatório. 

O conceito de heurística em CSP não é o mesmo usado na procura tradicional. Nesta, uma heurística é uma estimativa da menor distância de um estado ao objectivo. Em CSP a heurística é uma preferência. Neste caso, com o MRV, em vez de termos uma ordem fixa para a selecção das variáveis a afectar vamos preferir as que tenham menores domínios. 

A ideia por detrás desta heurística é que, provavelmente, iremos antecipar as falhas escolhendo variáveis com menores domínios, diminuindo a procura.

Notem que esta heurística poderia ser aplicada de forma estática uma única vez no início ou de forma dinâmica, sempre que se escolhe a próxima variável. Na sua versão dinâmica, não faz sentido no retrocesso sem inferência.


### Reformulando o CSP $A>B>C$.
Vamos imaginar que o nosso velho problema $A>B>C$ é ligeiramente diferente e que uma das variáveis: $C$, tem menos um elemento no domínio, i.e., não tem o 3.

In [ ]:
def CSP_cab ():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema ABC
    """
    
    #Definir Variáveis
    variaveis_abc = ['A', 'B','C']
    
    """Definir Domínios
    Devolve um dicionario com os domínios com as variáveis do problema dos Golfers.
    """   
    dominios_abc = {}
    dominios_abc['A'] = [1,2,3]
    dominios_abc['B'] = [1,2,3]
    dominios_abc['C'] = [1,2]


    #Definir Vizinhos    
    # parse_neighbors trata da simetria. do not worry
    vizinhos_abc = parse_neighbors('A : B; B: C')

    #Definir função que verifica restrições binárias
    def restricoes_abc(X, a, Y, b) :
        """
        A implementação em CSP assume que as restrições são binárias.
        Retorna True se (X=a,Y=b) satisfaz as restrições entre X e Y ou se não há restrição entre X e Y
        """
        out = True
        if (X == 'A' and Y=='B') or (X == 'B' and Y=='C'):
            out = a > b
        elif (X == 'B' and Y == 'A') or (X == 'C' and Y == 'B'):
            out = b > a
            
        return out
        
    return CSP(variaveis_abc, dominios_abc, vizinhos_abc, restricoes_abc)

Vamos aplicar o retrocesso sem inferência e sem heurísticas:

In [ ]:
p = CSP_cab()
r = backtracking_search(p)
print('Assignment p = ',r)

Vamos aplicar a heurística `mrv` para escolha de variáveis. Esta heurística dá preferências às variáveis com domínios mais pequenos. Isso irá ter implicações na ordem da escolha das variáveis para a afectação como podem comprovar.

In [ ]:
p = CSP_cab()
r = backtracking_search(p,select_unassigned_variable = mrv)
print('Assignment p = ',r)

Pressionemos o botão verbose!

In [ ]:
p = CSP_cab()
r = backtracking_search(p,select_unassigned_variable = mrv,verbose=True)
print('Assignment p = ',r)

A variável $C$ é a primeira a ser escolhida.. Depois tanto faz escolher a $A$ como a $B$, como podem comprovar já a seguir, ao repetirmos a resolução do problema 20 vezes.

In [ ]:
for i in range(20):
    p = CSP_cab()
    r = backtracking_search(p, select_unassigned_variable = mrv)
    print('Assignment = ',r) 

com a heurística `mrv`, as variáveis empatadas, com domínios de comprimento igual, são ordenadas de forma aleatória, mas $C$ é sempre escolhida primeiro, porque tem o menor domínio inicial.

### Utilização de heurísticas e inferência no BACKTRACKING-SEARCH

#### MRV + FORWARD-CHECKING

Notem que se utilizarmos a inferência forward checking em conjunto com a heurística de variável mais restringida (mrv), se $A$ for a primeira a ser escolhida então $B$ será sempre a segunda; se $C$ for a primeira variável então será sempre $B$ a segunda a ser escolhida. Quando é $B$ a primeira, então como $B$ está restringida com $C$ e com $A$, a segunda variável pode tanto ser $A$ como $C$.

In [ ]:
for i in range(20):
    p = CSP_abc()
    r = backtracking_search(p, select_unassigned_variable = mrv,inference=forward_checking)
    print('Assignment = ',r) 

#### Exercício 9
Para o problema das 20 raínhas teste o tempo de execução usando retrocesso com "forward checking" com e sem uso de MRVe também o retrocesso simples com mrv. 

In [ ]:
## Resolução exercício 9



## As palavras cruzadas
Consideremos agora o problema das palavras cruzadas, exercício da aula anterior. Relembremos esse problema.
<img src=".\imagens\PalavrasCruzadas2x2.png" alt="Drawing" style="width: 200px;"/>
Modelize um problema de palavras cruzadas usando CSP, em que temos uma grelha 2x2 e 8 palavras possíveis para dispor 4 delas nas 2 linhas e 2 colunas. Cada palavra só pode aparecer numa das 4 regiões possíveis (2 linhas e 2 colunas) e as letras das palavras têm de ser as mesmas quando as palavras se intersectam.

Vamos supor que temos 8 palavras possíveis para colocar: {MA, OS, AS, TU, LI, RO, TO, MO}.

### Formulação

#### Variáveis
Vamos considerar que temos 4 variáveis correspondentes às 4 palavras: $T$(opo), $B$(ase), $E$(squerda) e $D$(ireita).

#### Domínios
Vamos ter as 8 palavras como domínio de todas as 4 variáveis.

#### Restrições

Vamos ter 4 restrições:
        
        A palavra de Topo partilha a sua 1a letra com a 1a letra da palavra da Esquerda
        A palavra de Topo partilha a sua 2a letra com a 1a letra da palavra da Direita
        A palavra de Baixo partilha a sua 1a letra com a 2a letra da palavra da Esquerda
        A palavra de Baixo partilha a sua 2a letra com a 2a letra da palavra da Direita
        
        
Como as restrições dependem da ordem dos pares de variáveis, precisarems de facto de criar 8 restrições: entre $T$ e $E$ e também entre $E$ e $T$, entre $T$ e $D$ mas também entre $D$ e $T$, e as 4 análogas.

Ataquemos então a formulação no Python

In [ ]:
# Resolução das palavras cruzadas pequeninas

def CSP_cruz ():
    """
    Retorna um objecto da classe CSP inicializado com as variaveis, os dominios,
    os vizinhos e restrições do problema cruz
    """
    
    #Definir a lista com as Variaveis correspondentes às palavras horizontais
    # do T(opo) e da B(ase), da E(esquerda) e da D(ireita)
    variaveis = ['T', 'B','E','D']
    
    # Definir um dicionario com os Domínios (afinal todas as palavras possíveis) que são strings
   
    dominios = {}
    for v in variaveis :
        dominios[v] = ['MA', 'OS', 'AS', 'TU', 'LI', 'RO', 'TO', 'MO']
    
    #Definir Vizinhos    -- parse_neighbors trata da simetria. do not worry
    vizinhos = parse_neighbors('T : E D B; E: D B; D: B')

    #Definir funcao que verifica restricoes binarias
    def restricoes(X, a, Y, b) :
        
        """
        A implementacao em CSP assume que as restricoes são binarias.
        Retorna True se (X=a,Y=b) satisfaz as restricoes entre X e Y
        """
        def teste00(a,b):
            return a[0]==b[0]

        def teste01(a,b):
            return a[0]==b[1]

        def teste10(a,b):
            return a[1]==b[0]

        def teste11(a,b):
            return a[1]==b[1]
 
        # A palavra de topo partilha a sua letra 1 com a letra 1 da palavra da esquerda
        # A palavra de topo partilha a sua letra 1 com a letra 1 da palavra da direita
        # A palavra de baixo partilha a sua letra 1 com a letra 2 da palavra da esquerda
        # A palavra de baixo partilha a sua letra 2 com a letra 2 da palavra da direita
        # cuidar da condicao simetrica
        restricoes = {('T','E'):teste00, ('T','D'):teste10, ('B','E'):teste01, ('B','D'):teste11,
                      ('E','T'):teste00, ('D','T'):teste01, ('E','B'):teste10, ('D','B'):teste11}
        
        # É preciso que as variáveis na tabela sejam diferentes e satisfaçam a função associada
        # as que não partilham letras terão de ser apenas diferentes.
        if (X,Y) in restricoes.keys():
            return (restricoes[(X,Y)](a,b) and  a!= b)  # para alem de partilharem letras terao de ser palavras diferentes
        return a != b  # para os pares de variaveis que não estao na tabela, por exemplo, entre T e B
        
    return CSP(variaveis, dominios, vizinhos, restricoes)

Criemos um problema CSP de palavras cruzadas

In [ ]:
csp = CSP_cruz()

Vamos procurar a solução usando a Profundidade com Retrocesso no modo frugal, sem forward checking.

In [ ]:
r = backtracking_search(csp)
print('Afectacoes p = ',r)

Apliquemos o retrocesso sem nenhuma inferência, no modo verbose

In [ ]:
csp.curr_domains = csp.domains
r = backtracking_search(csp,verbose=True)
print('Afectacoes p = ',r)

Apliquemos a versão "verbose" do retrocesso com filtragem por forward checking

In [ ]:
cruz = CSP_cruz()
r = backtracking_search(cruz,inference = forward_checking, verbose = True)
print('Afectações = ',r) 

Como vimos, ao afectar a T a palavra "MO", o domínio de B não foi afectado porque não existem restrições envolvendo T e B, mas as outras variáveis, E e D, viram os respectivos domínios reduzidos apenas a 1 valor.
Quando se faz a afectação de B a OS, os restantes domínios, de E e de D mantêm-se. A seguir vemos que a solução é directa, pois E só pode tomar o valor MO que por forward checking não afecta o domínio de D que tomará o seu único valor: AS. 

Vamos executar o AC3 para o problema das palavras cruzadas

In [ ]:
#Execução do AC3
cruz = CSP_cruz()
p_ac3 = AC3(cruz)
print("Domínios após AC3 = ", p_ac3.curr_domains)

Houve uma redução substancial dos domínios de cada uma das 4 variáveis. Cada domínio passou de 8 para 2 palavras.
Agora acompanhando os passos do algoritmo, se tiverem paciência claro.

In [ ]:
#Execução do AC3
cruz = CSP_cruz()
p_ac3 = AC3(cruz,verbose=True)
print("Domínios após ACP = ", p_ac3.curr_domains)

Uma boa utilização do AC3 é no início e depois do pré-processamento aplicar a procura com retrocesso com ou sem forward-checking.
Vamos fazê-lo:

### Pré-processamento com o AC3 seguido de procura com e sem inferência por forward checking

Continuando com o problema das palavras cruzadas, apliquemos primeiro o AC3 e depois a procura básica sem inferência.

In [ ]:
#Execução do AC3
p = CSP_cruz()
p_ac3 = AC3(p)
print("Domínios após AC3 = ", p_ac3.curr_domains)
r = backtracking_search(p_ac3, verbose = False)
print('Assignment = ',r) 

Agora em modo verbose:

In [ ]:
#Execução do AC3
p = CSP_cruz()
p_ac3 = AC3(p)
print("Domínios após AC3 = ", p_ac3.curr_domains)
r = backtracking_search(p_ac3, verbose = True)
print('Assignment = ',r) 

Podemos agora aplicar o retrocesso com forward checking após o pré-processamento através do AC3:

In [ ]:
#Execução do AC3
p = CSP_cruz()
p_ac3 = AC3(p)
print("Domínios após AC3 = ", p_ac3.curr_domains)
r = backtracking_search(p_ac3, inference=forward_checking,verbose = True)
print('Assignment = ',r) 

### Procura com inferência em que se mantém a consistência (MAC)
Para além de aplicarmos o AC3 no início podemos também invocá-lo como inferência logo após tentarmos fazer uma afectação. Se o AC3 falhar, i.e. algum dos domínios de variáveis por afectar ficar vazio, então tenta-se nova afectação de valor à variável. Se todos os valores falharem, retrocede-se, desfazendo a última afectação e tentando um novo valor...

o campo inference terá o valor mac

* **MAC** (Mantain Arc Consistenty, usando o AC3): Utilizando o **AC3** após cada afectação da variável. Mais caro computacionalmente


Vamos aplicar a profundidade com retrocesso utilizando o AC3 para inferência, i.e., mantendo a consistência dos arcos, ao problema das palavras cruzadas.

In [ ]:
pz = CSP_cruz()
p_ac3 = AC3(p)
print("Domínios após AC3 = ", p_ac3.curr_domains)
r = backtracking_search(p_ac3,inference = mac)
print('Afectações r = ',r)

Vamos fazê-lo, em modo verboso, para ver tudo o que se passa.
Notem que não aplicamos o pré-processamento com o AC3, e a consistência dos arcos é apenas mantida após cada afectação de variáveis.

In [ ]:
cruz = CSP_cruz()
cruz_ac3 = AC3(cruz)
print("Domínios após AC3 = ", p_ac3.curr_domains)
r = backtracking_search(cruz_ac3,inference = mac, verbose = True)
print('Assignment = ',r) 

#### Exercício 10
Para o problema das 50 raínhas compare o tempo de execução usando retrocesso com "forward checking" com pré-filtragem pela consistência dos arcos e a versão MAC. Utilizem a verão feita no exercíco X em que usam uma ordem aleatória para as variáveis e em que utilizam MRV.

In [ ]:
# resolução do exercício 10



### Exercício 11 - Problema dos Jogadores de Golfe (Golfers)

Teste a profundidade com retrocesso, sem inferência, com as duas inferências e com a heurística mrv, isoladamente ou de forma combinada. Verifique que a utilização simultânea de heurísticas para selecção das variáveis a testar primeiro e de inferência para detectar falhas antecipadamente permite obter uma solução num menor número de passos.


In [ ]:
# Resolução do exercício 11



### Exercício 12 - Problema da coloração de mapas (Map Coloring)

<img src=".\imagens\mapa-colorir.PNG" alt="Drawing" style="width: 200px;"/>

O problema de colorir o mapa usando 4 cores pode ser modelizado como um csp da forma seguinte:

```python
vars=['A','B','C','D','E']
cores=list('RBGY')
dominios={}
for v in vars:
    dominios[v]=cores
mapa = CSP(vars, dominios, parse_neighbors('A: B C D; B: C D; C: D; D: E; E: '),different_values_constraint)

```

Utilize o algoritmo `BACKTRACKING-SEARCH` para o resolver tal como fez para o Golfers.

Verifique que neste problema o AC3 não reduz os domínios das variáveis.

Verifique que neste caso são obtidas soluções diferentes (diferentes possibilidades de coloração) quando se usa ou não heurísticas ou inferência:

In [ ]:
# Resolução do exercício 12



### Exercício 13 - Junta Panelas
 Suponha que o João e a Joana querem organizar uma festa com os
seus amigos: a Alice, o Roberto, o Carlos e o Duarte. Será uma festa do género “Junta Panelas”
em que todos convidados trazem comes e bebes. Neste caso, os organizadores irão tratar das
bebidas e, para organizar a festa, eles irão dar a cada um dos seus amigos uma folha de papel
para registarem o número de pratos que irão trazer. No entanto, os seus amigos não assinaram
as folhas respectivas e o João e a Joana têm dificuldade em perceber a letra e a autoria de cada
uma das folhas. No entanto, eles são capazes de perceber o suficiente para obterem as
restrições seguintes:

    1. Cada um dos amigos traz no mínimo 1 prato e no máximo 6.
    2. Ninguém traz mais pratos do que a Alice.
    3. O Roberto traz um número ímpar de pratos.
    4. O Carlos traz o dobro dos pratos do Roberto.
    5. O Duarte traz exactamente mais dois pratos do que o Carlos.

a) Modelize o Junta Panelas como um Problema de Satisfação de Restrições uando a classe CSP, levando em conta a filtragem inicial das restrições unitárias.  
b)  Mostre como ficam os domínios depois depois de forçarmos a consistência dos arcos Alice $\rightarrow$ Carlos, Alice $\rightarrow$ Duarte e Alice $\rightarrow$ Roberto?  
c)  Qual é o resultado se aplicarmos o “forward checking”, partindo dos domínios iniciais depois de satisfazerem as restrições unitárias e afectarmos o valor de 4 para a Alice. 

In [ ]:
# Solução do exercício 13

